In [1]:
import gensim.models.word2vec as w2v
import pickle
import multiprocessing
import re
import ast
import nltk
import time

import pandas as pd
import numpy as np

import lasagne
import theano
import theano.tensor as T

In [2]:
# data = open('datasetV2twplustag.txt').read().split('\n')
# data = pd.Series(data)

In [3]:
with open('twitter_df.pickle', 'rb') as dfile:
    raw_data = pickle.load(dfile)
labels = raw_data['tags']
labels = pd.Series([ast.literal_eval(label)[0] for label in labels])
raw_data = raw_data['tokenized_text']

In [4]:
t2v = w2v.Word2Vec.load('tweet2vec.w2v')

In [5]:
sentences = []
for row in raw_data:
    for sentence in row:
        sentences.append(sentence)

In [6]:
data = raw_data.apply(lambda row: [[t2v.wv[word] for word in sentence] for sentence in row])
data = np.array([np.array(batch[0]) for batch in data])

In [7]:
labs, counts = np.unique(labels, return_counts=True)

In [8]:
counts, labs = zip(*sorted(zip(counts, labs), reverse=True))

In [9]:
split = round(0.1*len(labs))
keep = labs[:split]
remove = labs[split:]

In [10]:
labels.shape

(279395,)

In [11]:
labels2 = pd.Series([lab if lab in keep else None for lab in labels])

In [12]:
y = np.array([t2v.wv[word] if word is not None else None for word in labels2])

In [13]:
mask = np.array([el is not None for el in y])

In [14]:
y = y[mask]
# data = data[mask]

In [15]:
y = np.vstack(y)

In [16]:
y.shape

(169086, 600)

In [17]:
data_size = labels.shape[0]

border = round(data_size*0.8)
X_train = data[:border]
X_test = data[border:]
y_train = y[:border]
y_test = y[border:]

In [19]:
with open('data.pickle', 'wb') as dfile:
    pickle.dump(zip(data), dfile)

MemoryError: 

In [158]:
y_train.shape

(169086, 600)

In [159]:
input_var = T.tensor3('inputs')
target_var = T.matrix('targets')

In [160]:
#Lasagne Seed for Reproducibility
lasagne.random.set_rng(np.random.RandomState(1))

# Sequence Length
SEQ_LENGTH = 7

# Number of units in the two hidden (LSTM) layers
N_HIDDEN = 40

# Optimization learning rate
LEARNING_RATE = .1

# All gradients above this will be clipped
GRAD_CLIP = 100

# How often should we check the output?
PRINT_FREQ = 500

# Number of epochs to train the net
NUM_EPOCHS = 50

# Batch Size
BATCH_SIZE = 100

# Number of classes
CLASS_NUMBER = len(keep)

In [26]:
def build_nn(input_var=None):
    print("Building network ...")

    # First, we build the network, starting with an input layer
    # Recurrent layers expect input of shape
    # (batch size, SEQ_LENGTH, num_features)    
    l_in = lasagne.layers.InputLayer(shape=(None, 1, 600), input_var=input_var)
    
#     l_emb = lasagne.layers.EmbeddingLayer(l_in, 600, )

    #Two stacked LSTM layers
    l_forward_1 = lasagne.layers.LSTMLayer(
        l_in, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

    l_forward_2 = lasagne.layers.LSTMLayer(
        l_forward_1, N_HIDDEN, grad_clipping=GRAD_CLIP,
        nonlinearity=lasagne.nonlinearities.tanh)

    #Slice the output from the LSTM layers to only take the final prediction
    l_forward_slice = lasagne.layers.SliceLayer(l_forward_2, -1, 1)

    l_out = lasagne.layers.DenseLayer(l_forward_2, num_units=CLASS_NUMBER, 
                                      W = lasagne.init.Normal(), nonlinearity=lasagne.nonlinearities.softmax)

    return l_out

In [96]:
network = build_nn(input_var)

Building network ...


In [97]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.categorical_crossentropy(prediction, target_var)
loss = loss.mean()

/home/daria/anaconda3/lib/python3.5/site-packages/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [98]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.01, momentum=0.9)

In [99]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.categorical_crossentropy(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

/home/daria/anaconda3/lib/python3.5/site-packages/theano/scan_module/scan.py:1019: Warning: In the strict mode, all neccessary shared variables must be passed as a part of non_sequences
  'must be passed as a part of non_sequences', Warning)


In [100]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [101]:
train_fn = theano.function([input_var, target_var], loss, updates=updates)

In [102]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [103]:
# def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
#     assert len(inputs) == len(targets)
#     if shuffle:
#             indices = np.arange(len(inputs))
#             np.random.shuffle(indices)
#     for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
#         if shuffle:
#             excerpt = indices[start_idx:start_idx + batchsize]
#         else:
#             excerpt = slice(start_idx, start_idx + batchsize)
#         inp = inputs[excert]
#         yield inputs[excerpt], targets[excerpt]

In [161]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
            indices = np.arange(len(inputs))
            np.random.shuffle(indices)
    for start_idx in range(len(inputs)):
        if shuffle:
            excerpt = indices[start_idx]
            inp = inputs[excerpt]
        else:
            inp = inputs[start_idx]
        inp = np.expand_dims(inp, 1)
#         inp = np.array(inp, dtype=np.float32)
        yield inp, targets[excerpt]

In [162]:
for epoch in range(NUM_EPOCHS):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 500, shuffle=True):
        inputs, targets = batch
        print(inputs.shape)
        train_err += train_fn(inputs, targets, allow_input_downcast=True)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 500, shuffle=False):
        inputs, targets = batch
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, NUM_EPOCHS, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

(5, 1, 600)


TypeError: ('Bad input argument to theano function with name "<ipython-input-101-a9a5249efc94>:1"  at index 1(0-based)', 'TensorType(int32, vector) cannot store a value of dtype float32 without risking loss of precision. If you do not mind this loss, you can: 1) explicitly cast your data to int32, or 2) set "allow_input_downcast=True" when calling "function".', array([ -5.93071512e-04,   6.51585637e-04,   6.13727025e-04,
         8.17847787e-04,   8.39702043e-05,   7.50090345e-04,
        -3.38226731e-04,  -7.38658928e-05,   8.30631936e-04,
        -1.04247360e-04,   7.12678535e-04,  -6.22358813e-04,
        -1.48864056e-04,   3.36624740e-04,   6.42150408e-04,
        -2.52210099e-04,   1.92125517e-04,  -6.03689230e-04,
         3.27086018e-04,   2.16319531e-04,  -1.31946654e-04,
        -2.04107389e-04,  -5.46766212e-04,   9.32203402e-05,
         3.04789661e-04,   1.21234392e-04,  -8.76966806e-05,
        -5.90709737e-04,   6.31688803e-04,   7.49788422e-04,
        -5.90504205e-04,  

In [95]:
X_train[0].shape

(7, 600)